# Wikidata にある日本の市区町村データの整合性チェック

Wikidata にある日本の市区町村データと、総務省サイトにあるそれとを比較し、一致することを確認する。

In [14]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, XML

# 表のすべての行を表示する
pd.set_option('display.max_rows', None)

In [2]:
# 総務省のサイト https://www.soumu.go.jp/denshijiti/code.html から
# 最新の「都道府県コード及び市区町村コード」を読み込む。
excel_file_url = "https://www.soumu.go.jp/main_content/000894847.xls"

# １枚目のシート。市町村と東京特別区。普通の区は含まない。
df_municipalities_sub = pd.read_excel(excel_file_url, sheet_name=0)
# カラム名を変更
df_municipalities_sub.rename(columns={'都道府県名\n（漢字）': '都道府県名', '市区町村名\n（漢字）': '市区町村名'}, inplace=True)
# 市区町村名のカラムが　NaN の行は都道府県なので削除する
df_municipalities_sub.dropna(subset=['市区町村名'], inplace=True)
# 市
df_cities = df_municipalities_sub.query('市区町村名.str.endswith("市")')
# 東京特別区
df_tokyo_wards = df_municipalities_sub.query('市区町村名.str.endswith("区")')
# 町
df_towns = df_municipalities_sub.query('市区町村名.str.endswith("町")')
# 村
df_villages = df_municipalities_sub.query('市区町村名.str.endswith("村")')

# ２枚目のシート。普通の区。
df_wards = pd.read_excel(excel_file_url, sheet_name=1)
# カラム名を変更
df_wards.rename(columns={'都道府県名\n（漢字）': '都道府県名', '市区町村名\n（漢字）': '市区町村名', '都道府県名\n（ｶﾅ）': '都道府県名\n（カナ）', '市区町村名\n（ｶﾅ）': '市区町村名\n（カナ）'}, inplace=True)
# 市区町村名のカラムが「市」で終わる行は市なので削除する
df_wards = df_wards.query('not 市区町村名.str.endswith("市")')

print('市の数: ' + str(len(df_cities)))
print('東京特別区の数: ' + str(len(df_tokyo_wards)))
print('町の数: ' + str(len(df_towns)))
print('村の数: ' + str(len(df_villages)))
print('区の数: ' + str(len(df_wards)))

# 二つを合わせてすべての自治体のリストを得る
#df_municipalities = pd.concat([df_municipalities_sub, df_wards],ignore_index=True)
#df_municipalities.sort_values('団体コード', ascending=True, inplace=True)
#df_municipalities.reset_index(inplace=True, drop=True)
#df_municipalities

市の数: 792
東京特別区の数: 23
町の数: 743
村の数: 189
区の数: 171


市の数: 792
東京特別区の数: 23
町の数: 743
村の数: 189（北方領土の6村を含む）
区の数: 171（2024/1/1 の浜松市の区統合（7区→3区）後の数を先取りしている。現在の175-7+3=171）

が正しい。（2023/10/03 現在）

In [4]:
# Wikidata から SPARQL 言語による検索結果を RDF(JSON) で得る関数を定義する。
def QuerySparql(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

## 都道府県ごとの市の数のチェック

In [30]:
# 都道府県ごとの市の数
num_cities_query = """
SELECT ?pref ?prefLabel ?prefCode (COUNT(DISTINCT ?city) AS ?count) WHERE {
  ?city wdt:P31/wdt:P279* wd:Q494721 . # ?city は「日本の市」またはその部分集合（の部分集合……）の要素である。
  MINUS { ?city wdt:P576 ?abolished. } # ただし「廃止日」がないこと。
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?city wdt:P131+ ?pref. # ?pref に位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  OPTIONAL { ?pref wdt:P429 ?prefCode. } # 都道府県の全国地方公共団体コード
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "ja" .
  }
}
GROUP BY ?pref ?prefLabel ?prefCode
ORDER BY ASC(?prefCode)
"""
json_num_cities_wikidata = QuerySparql(num_cities_query)

# pandas DataFrame へ変換
num_cities_wikidata = []
for c in json_num_cities_wikidata:
    city = {
        '都道府県名': c['prefLabel']['value'],
        '市の数': int(c['count']['value'])
    }
    num_cities_wikidata.append(city)
df_num_cities_wikidata = pd.DataFrame(num_cities_wikidata)
df_num_cities_wikidata

,都道府県名,市の数
0,北海道,35
1,青森県,10
2,岩手県,14
3,宮城県,14
4,秋田県,13
5,山形県,13
6,福島県,13
7,茨城県,32
8,栃木県,14
9,群馬県,12


これらの Wikidata による市の数を、総務省のページ
https://www.j-lis.go.jp/spd/code-address/kenbetsu-inspection/cms_11914151.html
にある数と比べ、一致することを確認すること。

In [5]:
# 市の一覧を Wikidata から取得
cities_query = """
SELECT DISTINCT ?pref ?prefLabel ?city ?cityLabel ?code WHERE {
  {
    # 以下のような例を得るために、wdt:P31 のあとに "/wdt:P279*" が必要。
    # 山形市 P31 中核市 P279 日本の市
    # 所沢市 P31 特例市 P279 日本の市
    ?city wdt:P31/wdt:P279* wd:Q494721 . # ?city は「日本の市」の（直接または間接的な）要素である。
  } 
  MINUS { ?city wdt:P576 ?abolished . } # ただし「廃止日」がないこと。

  OPTIONAL { ?city wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?city wdt:P131+ ?pref. # ?pref に（直接または間接的に）位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
    MINUS { ?pref wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_cities_wikidata = QuerySparql(cities_query)

# pandas DataFrame へ変換
cities_wikidata = []
for item in json_cities_wikidata:
    city = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None),
        '市名': (item['cityLabel']['value'] if ('cityLabel' in item) else None)
    }
    cities_wikidata.append(city)
df_cities_wikidata = pd.DataFrame(cities_wikidata)
df_cities_wikidata

,団体コード,都道府県名,市名
0,11002,北海道,札幌市
1,12025,北海道,函館市
2,12033,北海道,小樽市
3,12041,北海道,旭川市
4,12050,北海道,室蘭市
...,...,...,...
787,472115,沖縄県,沖縄市
788,472123,沖縄県,豊見城市
789,472131,沖縄県,うるま市
790,472140,沖縄県,宮古島市


In [6]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
df_cities_only_in_wikidata = df_cities_wikidata[~df_cities_wikidata['団体コード'].isin(df_cities['団体コード'])]
df_cities_only_in_wikidata

,団体コード,都道府県名,市名


In [7]:
# 総務省のデータにあって Wikidata にはないものを表示
# 何もないのが正しい。
df_cities_not_in_wikidata = df_cities[~df_cities['団体コード'].isin(df_cities_wikidata['団体コード'])]
df_cities_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）


In [9]:
# 東京特別区の一覧を Wikidata から取得
tokyo_wards_query = """
SELECT DISTINCT ?pref ?prefLabel ?ward ?wardLabel ?code WHERE {
  ?ward wdt:P31 wd:Q5327704 . # ?ward は「日本の特別区」の要素である。
  MINUS { ?ward wdt:P576 ?abolished . } # ただし「廃止日」がないこと。

  OPTIONAL { ?ward wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?ward wdt:P131+ ?pref. # ?pref に（直接または間接的に）位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
    MINUS { ?pref wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_tokyo_wards_wikidata = QuerySparql(tokyo_wards_query)

# pandas DataFrame へ変換
tokyo_wards_wikidata = []
for item in json_tokyo_wards_wikidata:
    ward = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None),
        '区名': (item['wardLabel']['value'] if ('wardLabel' in item) else None)
    }
    tokyo_wards_wikidata.append(ward)
df_tokyo_wards_wikidata = pd.DataFrame(tokyo_wards_wikidata)
df_tokyo_wards_wikidata

,団体コード,都道府県名,区名
0,131016,東京都,千代田区
1,131024,東京都,中央区
2,131032,東京都,港区
3,131041,東京都,新宿区
4,131059,東京都,文京区
5,131067,東京都,台東区
6,131075,東京都,墨田区
7,131083,東京都,江東区
8,131091,東京都,品川区
9,131105,東京都,目黒区


In [10]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
df_tokyo_wards_only_in_wikidata = df_tokyo_wards_wikidata[~df_tokyo_wards_wikidata['団体コード'].isin(df_tokyo_wards['団体コード'])]
df_tokyo_wards_only_in_wikidata

,団体コード,都道府県名,区名


In [11]:
# 総務省のデータにあって Wikidata にはないものを表示
# 何もないのが正しい。
df_tokyo_wards_not_in_wikidata = df_tokyo_wards[~df_tokyo_wards['団体コード'].isin(df_tokyo_wards_wikidata['団体コード'])]
df_tokyo_wards_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）


In [15]:
# 区の一覧を Wikidata から取得
wards_query = """
SELECT DISTINCT ?pref ?prefLabel ?city ?cityLabel  ?ward ?wardLabel ?code WHERE {
  {?ward wdt:P31 wd:Q137773 . } # ?ward は「日本の区」の要素である。
  MINUS { ?ward wdt:P576 ?abolished . } # ただし「廃止日」がないこと。

  OPTIONAL { ?ward wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    ?ward wdt:P131 ?city. # ?ward は ?city に位置する。
    ?city wdt:P31 wd:Q494721. # ?city は「日本の市」である。
    MINUS { ?city wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  }
  OPTIONAL {
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?ward wdt:P131+ ?pref. # ?pref に（直接または間接的に）位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
    MINUS { ?pref wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_wards_wikidata = QuerySparql(wards_query)

# pandas DataFrame へ変換
wards_wikidata = []
for item in json_wards_wikidata:
    ward = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None),
        '市名': (item['cityLabel']['value'] if ('cityLabel' in item) else None),
        '区名': (item['wardLabel']['value'] if ('wardLabel' in item) else None)
    }
    wards_wikidata.append(ward)
df_wards_wikidata = pd.DataFrame(wards_wikidata)
df_wards_wikidata

,団体コード,都道府県名,市名,区名
0,11011,北海道,札幌市,中央区
1,11029,北海道,札幌市,北区
2,11037,北海道,札幌市,東区
3,11045,北海道,札幌市,白石区
4,11053,北海道,札幌市,豊平区
5,11061,北海道,札幌市,南区 (札幌市)
6,11070,北海道,札幌市,西区 (札幌市)
7,11088,北海道,札幌市,厚別区
8,11096,北海道,札幌市,手稲区
9,11100,北海道,札幌市,清田区


浜松市の 2024/1/1 の区統合によってできる中央区と浜名区が含まれるので、
現在の実数 175 よりも 2 多い 177 行がヒットする。

In [16]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
#
# 2023/10/01 現在、浜松市の2024年に予定されている統合前の区7つがここに表示される。
df_wards_only_in_wikidata = df_wards_wikidata[~df_wards_wikidata['団体コード'].isin(df_wards['団体コード'])]
df_wards_only_in_wikidata

,団体コード,都道府県名,市名,区名
70,221317,静岡県,浜松市,中区
71,221325,静岡県,浜松市,東区
72,221333,静岡県,浜松市,西区
73,221341,静岡県,浜松市,南区
74,221350,静岡県,浜松市,北区
75,221368,静岡県,浜松市,浜北区
76,221376,静岡県,浜松市,天竜区


In [17]:
# 総務省のデータにあって Wikidata にはないものを表示
# 何もないのが正しい。
#
# 2023/10/01 現在、浜松市の2024年に予定されている統合で団体コードがつけ変わる天竜区がここに表示される。
df_wards_not_in_wikidata = df_wards[~df_wards['団体コード'].isin(df_wards_wikidata['団体コード'])]
df_wards_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）,Unnamed: 5
82,221406,静岡県,浜松市天竜区,ｼｽﾞｵｶｹﾝ,ﾊﾏﾏﾂｼﾃﾝﾘｭｳｸ,R6.1.1施行


In [19]:
# 町の一覧を Wikidata から取得
towns_query = """
SELECT DISTINCT ?pref ?prefLabel ?district ?districtLabel ?town ?townLabel ?code WHERE {
  ?town p:P31 ?statement2 .
  ?statement2 ps:P31 wd:Q1059478 . # ?town は「日本の町」である。
  MINUS { ?statement2 pq:P582 ?endTime. } # 「終了日」の修飾子がないこと。
  MINUS { ?town wdt:P576 ?abolished . } # 「廃止日」がないこと。

  OPTIONAL { ?town wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?town wdt:P131+ ?pref. # ?pref に（直接または間接的に）位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
    MINUS { ?pref wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  }
  OPTIONAL {
    ?town p:P131 ?statement1.
    ?statement1 ps:P131 ?district. # ?town は ?district に位置する。
    ?district wdt:P31 wd:Q1122846. # ?district は「郡」である。
    MINUS { ?statement1 pq:P582 ?endTime. } # 「終了日」が修飾子にあるものは除く。
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_towns_wikidata = QuerySparql(towns_query)

# pandas DataFrame へ変換
towns_wikidata = []
for item in json_towns_wikidata:
    town = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None),
        '郡名': (item['districtLabel']['value'] if ('districtLabel' in item) else None),
        '町名': (item['townLabel']['value'] if ('townLabel' in item) else None)
    }
    towns_wikidata.append(town)
df_towns_wikidata = pd.DataFrame(towns_wikidata)
df_towns_wikidata

,団体コード,都道府県名,郡名,町名
0,13030,北海道,石狩郡,当別町
1,13315,北海道,松前郡,松前町
2,13323,北海道,松前郡,福島町
3,13331,北海道,上磯郡,知内町
4,13340,北海道,上磯郡,木古内町
5,13374,北海道,亀田郡,七飯町
6,13439,北海道,茅部郡,鹿部町
7,13455,北海道,茅部郡,森町
8,13463,北海道,二海郡,八雲町
9,13471,北海道,山越郡,長万部町


In [20]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
df_towns_only_in_wikidata = df_towns_wikidata[~df_towns_wikidata['団体コード'].isin(df_towns['団体コード'])]
df_towns_only_in_wikidata

,団体コード,都道府県名,郡名,町名


In [21]:
# 総務省のデータにあって Wikidata にはないものを表示
# 何もないのが正しい。
df_towns_not_in_wikidata = df_towns[~df_towns['団体コード'].isin(df_towns_wikidata['団体コード'])]
df_towns_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）


In [22]:
# 村の一覧を Wikidata から取得
villages_query = """
SELECT DISTINCT ?pref ?prefLabel ?district ?districtLabel ?village ?villageLabel ?code WHERE {
  ?village p:P31 ?statement2 .
  ?statement2 ps:P31 wd:Q4174776 . # ?village は「日本の村」である。
  MINUS { ?statement2 pq:P582 ?endTime. } # 「終了日」の修飾子がないこと。
  MINUS { ?village wdt:P576 ?abolished . } # ただし「廃止日」がないこと。

  OPTIONAL { ?village wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?village wdt:P131+ ?pref. # ?pref に（直接または間接的に）位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
    MINUS { ?pref wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  }
  OPTIONAL {
    ?village p:P131 ?statement1.
    ?statement1 ps:P131 ?district. # ?village は ?district に位置する。
    ?district wdt:P31 wd:Q1122846. # ?district は「郡」である。
    MINUS { ?statement1 pq:P582 ?endTime. } # 「終了日」が修飾子にあるものは除く。
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_villages_wikidata = QuerySparql(villages_query)

# pandas DataFrame へ変換
villages_wikidata = []
for item in json_villages_wikidata:
    village = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None),
        '郡名': (item['districtLabel']['value'] if ('districtLabel' in item) else None),
        '町名': (item['villageLabel']['value'] if ('villageLabel' in item) else None)
    }
    villages_wikidata.append(village)
df_villages_wikidata = pd.DataFrame(villages_wikidata)
df_villages_wikidata

,団体コード,都道府県名,郡名,町名
0,13048,北海道,石狩郡,新篠津村
1,13919,北海道,島牧郡,島牧村
2,13960,北海道,虻田郡,真狩村
3,13978,北海道,虻田郡,留寿都村
4,14036,北海道,古宇郡,泊村
5,14044,北海道,古宇郡,神恵内村
6,14095,北海道,余市郡,赤井川村
7,14630,北海道,勇払郡,占冠村
8,14702,北海道,中川郡,音威子府村
9,14851,北海道,苫前郡,初山別村


In [23]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
df_villages_only_in_wikidata = df_villages_wikidata[~df_villages_wikidata['団体コード'].isin(df_villages['団体コード'])]
df_villages_only_in_wikidata

,団体コード,都道府県名,郡名,町名


In [24]:
# 総務省のデータにあって Wikidata にはないものを表示
# 何もないのが正しい。
df_Villages_not_in_wikidata = df_villages[~df_villages['団体コード'].isin(df_villages_wikidata['団体コード'])]
df_Villages_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）
